# Øving 4 - TMA4320
* **Veiledning:** Torsdag 13. februar, 2020, 0815-1000 i H3
* **Innleveringsfrist:** Torsdag 20. februar, 2020, kl 1400
* **Innleveringsmetode** Følgende to krav er nødvendig for godkjenning
    1. Opplasting av Jupyter Notebook (individuelt) i Blackboard
    2. Svare på Blackboardskjema for de tre kontrollspørsmålene i øvingen

Denne øvingen dreier seg om numerisk integrasjon. Ta en kopi av denne Notebook'en og fyll inn dine svar. Ta quizen i Bb.

**Oppgave 1**
Vi ser i første omgang på en numerisk integrasjonsformel på intervallet $[0,1]$ av formen

$$
    Q[f] = s\left(f(1)-f(0)\right) + b f(\alpha) 
$$

der $s$, $b$ og $\alpha$ er parametre som kan velges.


<div class="alert alert-block alert-success">
**Oppgave 1a.**
Finn verdier for $s$, $b$ og $\alpha$ som gjør at formelen får høyest mulig presisjonsgrad.
</div>


**Løsning 1a.**
Betingelser

$$
\begin{array}{llcl}
p=0: & s(1-1)+b=\int_0^1 1\;dx &\Rightarrow& b=1 \\ 
p=1: & s(1-0)+b\alpha=\int_0^1 x &\Rightarrow& s+b\alpha=\frac12 \\
p=2: & s(1^2-0^2)+b\alpha^2=\int_0^1 x^2 dx &\Rightarrow& s+b\alpha^2 =\frac13
\end{array}
$$

Så $b=1$ og differansen mellom de to siste betingelsene gir $\alpha^2-\alpha+\frac16=0$ som gir at $\alpha=\frac12\pm\frac16\sqrt{3}$ og dermed $s=\mp\frac16\sqrt{3}$. Vi får de to løsningene 

$$(b,\alpha,s)\in\left\{(1,\tfrac12+\tfrac16\sqrt{3},-\tfrac16\sqrt{3}),\
(1,\tfrac12-\tfrac16\sqrt{3},\tfrac16\sqrt{3})\right\}$$

Ved konstruksjon har vi med disse to løsningene funnet formler med presisjonsgrad 2. Ved å sette løsningene inn betingelsen for $p=3$ finner vi at presisjonsgraden ikke er større enn 2.

**Kontrollspørsmål 1:**
Hva er den største mulige presisjonsgrad? **Svar:** $2$

**Kontrollspørsmål 2:** Det fins to løsninger for $\alpha$, hva er verdien av den minste av disse?
**Svar:** $\tfrac12-\tfrac16\sqrt{3}$



Vi modifiserer nå formelen ved å transformere den til intervallet $[x_k,x_k+h]$.
Et lineært variabelskifte, $x=x_k+th$, gir oss

$$
  \int_{x_k}^{x_k+h}f(x)\,dx = h\int_0^1 f(x_k+th)\,dt 
$$

som gir den modifiserte formelen

$$
     Q_{[x_k,x_k+h]}[f] = h\big(s(f(x_k+h)-f(x_k)) + b f(x_k+\alpha h)\big)
$$

Neste skritt er å lage en sammensatt formel; definer $x_k=a+kh$ der $h=\frac{b-a}{n}$ og $n$ er antall intervaller. Vi kan nå lage den sammensatte formelen

$$
  \bar{Q}_n[f] = \sum_{k=0}^{n-1} Q_{[x_k,x_k+h]}[f]
$$

<div class="alert alert-block alert-success">
**Oppgave 1b.** Skriv opp formelen for $\bar{Q}_n[f]$ med de verdiene du har funnet for parametrene $b, \alpha, s$ i forrige oppgave.
</div>

**Løsning 1b.** En påtenkt kansellering skjer her, og man kan skrive

$$
   \bar{Q}_n[f] = hs(f(b)-f(a)) + h \sum_{k=0}^{n-1} f(x_k+\alpha h)  
$$

der $s$ og $\alpha$ tas fra løsningen av **(1a)**.



**Kontrollspørsmål 3:**
Hvor mange funksjonsevalueringer (beregninger av $f(x)$) trenger man med $n$ intervaller? **Svar:** n+2


<div class="alert alert-block alert-success">
**Oppgave 1c.**
Implementer metoden i Python. Anvend den på integralet

$$
I =   \int_{-2}^2 \frac{\sin x}{x}\,dx
$$

med $n=4,8,16,32,64,128$ og sammenlign svaret med sammensatt trapesmetode med samme antall intervaller.
Husk unntaksbehandling av $x=0$ (spesielt relevant for trapes).
</div>

**Løsning 1c.** 

In [1]:
import numpy as np

def f(x):
    if x==0:
        return 1
    else:
        return np.sin(x)/x

def teleformula(f,a,b,n,c):
    '''Beregner approksimasjon til I basert på formelen utledet i denne oppgaven
    f: integranden
    a,b: Intervallgrenser
    n: Antall intervaller
    c: skal være 1 eller -1 og velger hvilken av de to metodene med presisjonsgrad 2 
        som skal brukes
    '''
    s=c*1./6.*np.sqrt(3)
    alpha=0.5-s
    h=(b-a)/n
    result=s*(f(b)-f(a))
    for k in range(n):
        result += f(a+(k+alpha)*h)
    return h*result

def trapesformula(f,a,b,n):
    h=(b-a)/n
    result=-(f(a)+f(b))/2.0
    for k in range(n+1):
        result += f(a+k*h)
    return h*result

na=[4,8,16,32,64,128,256]
I = 3.2108259536053896972  # Har funnet 'eksakt svar' for sammenligning
c=-1 #valg av metode, bytt til c=-1 for å få den andre løsningen
print('{:^24}  {:^24}'.format('Feil i ny formel','Feil i trapes'))
print('{:^12} {:^12} {:^12} {:^12}'.format('Feil','Feil/n^4','Feil','Feil/n^2'))
for n in na: 
    Q=teleformula(f,-2.,2.,n,c)
    Qt=trapesformula(f,-2.,2.,n)
    err=np.abs(Q-I)
    errT=np.abs(Qt-I)
    print('{:12.8f} {:12.8f} {:12.8f} {:12.8f}'.format(err,err*n**4,errT,errT*n**2))
        
    


    Feil i ny formel           Feil i trapes      
    Feil       Feil/n^4       Feil       Feil/n^2  
  0.00011210   0.02869697   0.07323527   1.17176433
  0.00000689   0.02823414   0.01818288   1.16370414
  0.00000043   0.02812060   0.00453797   1.16171959
  0.00000003   0.02809235   0.00113401   1.16122532
  0.00000000   0.02808531   0.00028347   1.16110187
  0.00000000   0.02808332   0.00007087   1.16107102
  0.00000000   0.02808189   0.00001772   1.16106330


Vi ser altså her at feilen i den nye formelen oppfører seg som $C h^4$, dette skyldes at integranden er symmetrisk.

**Kontrollspørsmål 4.** Fra eksperimentene kan man estimere at feilen i den nye formelen på dette problemet oppfører seg som $|I-\bar{Q}_n[f]|\approx C n^q$ for et heltall $q$. Bestem $q$.
**Svar** $q=4$